In [8]:
import numpy as np
import pandas as pd

import os
import random
import re
import sys

In [9]:
DAMPING = 0.85
SAMPLES = 10000

In [10]:
def main():
    if len(sys.argv) != 2:
        sys.exit("Usage: python pagerank.py corpus")
    corpus = crawl(sys.argv[1])
    ranks = sample_pagerank(corpus, DAMPING, SAMPLES)
    print(f"PageRank Results from Sampling (n = {SAMPLES})")
    for page in sorted(ranks):
        print(f"  {page}: {ranks[page]:.4f}")
    ranks = iterate_pagerank(corpus, DAMPING)
    print(f"PageRank Results from Iteration")
    for page in sorted(ranks):
        print(f"  {page}: {ranks[page]:.4f}")

In [11]:
def crawl(directory):
    """
    Parse a directory of HTML pages and check for links to other pages.
    Return a dictionary where each key is a page, and values are
    a list of all other pages in the corpus that are linked to by the page.
    """
    pages = dict()

    # Extract all links from HTML files
    for filename in os.listdir(directory):
        if not filename.endswith(".html"):
            continue
        with open(os.path.join(directory, filename)) as f:
            contents = f.read()
            links = re.findall(r"<a\s+(?:[^>]*?)href=\"([^\"]*)\"", contents)
            pages[filename] = set(links) - {filename}

    # Only include links to other pages in the corpus
    for filename in pages:
        pages[filename] = set(
            link for link in pages[filename]
            if link in pages
        )

    return pages

In [12]:
def transition_model(corpus, page, damping_factor):
    """
    Return a probability distribution over which page to visit next,
    given a current page.

    With probability `damping_factor`, choose a link at random
    linked to by `page`. With probability `1 - damping_factor`, choose
    a link at random chosen from all pages in the corpus.
    """

    total_pages = len(corpus)
    prob = {}

    # Case 1: If page has no outgoing links, assume equal probability for all pages

    if not corpus[page]:
        for p in corpus:
            prob[p] = 1/total_pages
        return prob
    
    # Case 2: If page has outgoing links, calculate probability distribution

    linked_pages = corpus[page]
    num_links = len(linked_pages)

    for p in corpus:
        prob[p] = (1-damping_factor)/total_pages
        
        if p in linked_pages:
            prob[p] += damping_factor/num_links

    round_prob = {k: round(v, 3) for k, v in prob.items()}
    return round_prob

corpus = {
    "1.html": {"2.html", "3.html"},
    "2.html": {"3.html"},
    "3.html": {"2.html"}
}

print(transition_model(corpus, "1.html", 0.85))


{'1.html': 0.05, '2.html': 0.475, '3.html': 0.475}


In [13]:
def sample_pagerank(corpus, damping_factor, n):
    """
    Return PageRank values for each page by sampling `n` pages
    according to transition model, starting with a page at random.

    Return a dictionary where keys are page names, and values are
    their estimated PageRank value (a value between 0 and 1). All
    PageRank values should sum to 1.
    """

    page_rank = {page: 0 for page in corpus}
    current_page = random.choice(list(corpus.keys()))

    for i in range(n):
        prob = transition_model(corpus, current_page, damping_factor)
        j = random.choices(list(prob.keys()), weights=prob.values())[0]
        page_rank[j] += 1/n
        current_page = j

    round_page_rank = {k: round(v, 3) for k, v in page_rank.items()}
    return round_page_rank

corpus = {
    "1.html": {"2.html", "3.html"},
    "2.html": {"3.html"},
    "3.html": {"2.html"}
}

print(sample_pagerank(corpus, 0.85, 10000))

{'1.html': 0.047, '2.html': 0.476, '3.html': 0.477}


In [ ]:
def iterate_pagerank(corpus, damping_factor):
    """
    Return PageRank values for each page by iteratively updating
    PageRank values until convergence.

    Return a dictionary where keys are page names, and values are
    their estimated PageRank value (a value between 0 and 1). All
    PageRank values should sum to 1.
    """

    num_pages = len(corpus)
    pagerank = {page: 1/num_pages for page in corpus}
    j = {page: 0 for page in corpus}

    tolerance = 0.001

    while True:
        max_change = 0
        for page in corpus:
            rank_sum = 0
            for p in corpus:

                # CASE 2: Page has links
                if page in corpus[p]:
                    rank_sum += pagerank[p]/len(corpus[p])
                
                # CASE 1: Page has no links
                elif not corpus[p]:
                    rank_sum += pagerank[p]/num_pages
            
            j[page] = (1 - damping_factor)/num_pages + damping_factor * rank_sum
            max_change = max(max_change, abs(j[page] - pagerank[page]))
        
        pagerank = j.copy()
        
        if max_change < tolerance:
            break  
    
    return {page: round(pr, 3) for page, pr in pagerank.items()}

corpus = {
    "1.html": {"2.html", "3.html"},
    "2.html": {"3.html"},
    "3.html": {"2.html"}
}

print(iterate_pagerank(corpus, 0.85))

{'1.html': 0.333, '2.html': 0.333, '3.html': 0.333}
